In [ ]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt

In this notebook I practice several concepts related to linear regression and least squares. In order to do so I need a function to create data with linear trend. The following function creates a randomized dataset from the parameters of a linear regression model (slope and intercept.)

In [ ]:
SIZE = 50
def generate_random_linear_sample(slope, intercept, noise_std, x_init, x_end):
    x = np.random.uniform(x_init, x_end, SIZE)
    y = x*slope + intercept + np.random.normal(0, noise_std, SIZE)
    
    return (x,y)

In [ ]:
x1, y1 = generate_random_linear_sample(0.85, 12, 8, 10, 50)
x2, y2 = generate_random_linear_sample(-1.23, 5, 5, 20, 80)
x3, y3 = generate_random_linear_sample(0.02, 10, 1, 100, 120)
fig, ax = plt.subplots(1,3)
ax[0].scatter(x1, y1)
ax[1].scatter(x2, y2)
ax[2].scatter(x3, y3)
fig.set_figwidth(12)

### Example of moderate positive linear relationship

### Example of strong negative linear relationship

### Example of very weak linear relationship